<a href="https://colab.research.google.com/github/agemagician/ProtTrans/blob/master/Embedding/TensorFlow/Advanced/ProtBert-BFD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h3> Extracting protein sequences' features using ProtBert-BFD pretrained-model <h3>

<b>1. Load necessry libraries including huggingface transformers<b>

In [14]:
!pip install -q transformers

In [15]:
import tensorflow as tf
from transformers import TFBertModel, BertTokenizer,BertConfig
import re
import numpy as np

<b>2. Load the vocabulary and ProtBert-BFD Model</b>

In [16]:
tokenizer = BertTokenizer.from_pretrained("Rostlab/prot_bert_bfd", do_lower_case=False )

In [17]:
model = TFBertModel.from_pretrained("Rostlab/prot_bert_bfd", from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the ckeckpoint

<b>3. Create or load sequences and map rarely occured amino acids (U,Z,O,B) to (X)<b>

In [18]:
sequences_Example = ["A E T C Z A O","S K T Z P"]

In [19]:
sequences_Example = [re.sub(r"[UZOB]", "X", sequence) for sequence in sequences_Example]

<b>4. Tokenize, encode sequences and load it into the GPU if possibile<b>

In [20]:
ids = tokenizer.batch_encode_plus(sequences_Example, add_special_tokens=True, pad_to_max_length=True, return_tensors="tf")

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1773: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [21]:
input_ids = ids['input_ids']
attention_mask = ids['attention_mask']

<b>5. Extracting sequences' features and load it into the CPU if needed<b>

In [22]:
embedding = model(input_ids)[0]

In [23]:
embedding = np.asarray(embedding)

In [24]:
attention_mask = np.asarray(attention_mask)

<b>6. Remove padding ([PAD]) and special tokens ([CLS],[SEP]) that is added by ProtBert-BFD model<b>

In [25]:
features = [] 
for seq_num in range(len(embedding)):
    seq_len = (attention_mask[seq_num] == 1).sum()
    seq_emd = embedding[seq_num][1:seq_len-1]
    features.append(seq_emd)

In [26]:
print(features)

[array([[ 0.05551099, -0.10461281, -0.03254102, ...,  0.05091645,
         0.0431913 ,  0.10181017],
       [ 0.13895634, -0.04658355,  0.02193595, ...,  0.06942667,
         0.1476294 ,  0.0650388 ],
       [ 0.14610632, -0.0809287 , -0.12500374, ..., -0.03651187,
         0.02485547,  0.0797754 ],
       ...,
       [ 0.02349947, -0.01549821, -0.05685321, ..., -0.01342234,
         0.01704329,  0.0643108 ],
       [ 0.08129992, -0.10929591, -0.03022921, ...,  0.08717711,
         0.02061502,  0.05156738],
       [ 0.06197424, -0.06417827, -0.02039693, ..., -0.02796511,
         0.08840055,  0.07532725]], dtype=float32), array([[-0.12292586, -0.1095154 , -0.1050995 , ...,  0.08474061,
         0.10824808,  0.03636007],
       [ 0.11176239, -0.06970412, -0.04924221, ...,  0.02316407,
        -0.00177084, -0.0239673 ],
       [ 0.10432272, -0.13054034, -0.08879344, ..., -0.14225952,
        -0.01330808, -0.04152388],
       [-0.00434562, -0.0853354 , -0.05003072, ..., -0.16549371,
     